In [ ]:
!echo $VIRTUAL_ENV

In [ ]:
import yfinance as yf
import math

yf.config.debug.logging = False

### Live Asset info

In [ ]:
# With multiple ticker symbols



seaholdings = yf.Ticker("SE")

live_stock = { "ticker": seaholdings.ticker,
               "displayed_name": seaholdings.info['shortName'],
               "previous_price": seaholdings.info['open'],
                "current_price": seaholdings.info['currentPrice'],
                "regular_maret_price": seaholdings.info['regularMarketPrice'],
                "price_change": round(seaholdings.info['currentPrice'] - seaholdings.info['open'], 2),
                "price_change_percent": round((seaholdings.info['currentPrice'] - seaholdings.info['open']) / seaholdings.info['open'] * 100, 3)
              }
live_stock

In [ ]:
seaholdings.info

### Price Statistics

In [ ]:
# Get current price of SE
seaholdings.info['currentPrice']

In [ ]:
# This is the most efficient way to get last month's price data,
# as it only fetches the data that is needed for the use case,
#  without fetching the whole ticker information.
patrimonial_watchlist = ["SE", "FRAGUAB.MX", "AGED.L"]
indexed_watchlist = ["SPYM", "VGK", "VWO", "EWZ", "NAFTRACISHRS.MX"]

indexed_historics = yf.download(indexed_watchlist, period="1mo", interval="1d")
indexed_historics.tail(10)

In [ ]:
# Get minimum price in the last trailing 30 days for VGK
indexed_historics["Low"]["VGK"].min()

In [ ]:
# Get maximum price in the last trailing 30 days
indexed_historics["High"]["VGK"].max()

In [ ]:
# Get average price in the last trailing 30 days for VGK
indexed_historics["Close"]["VGK"].mean()

### Live Watchlist Current Prices (for pooling)

In [ ]:
indexed_prices = yf.Tickers(indexed_watchlist)

print(f" SPYM: {indexed_prices.tickers['SPYM'].info['regularMarketPrice']}")
print(f" VGK: {indexed_prices.tickers['VGK'].info['regularMarketPrice']}")
print(f" VWO: {indexed_prices.tickers['VWO'].info['regularMarketPrice']}")
print(f" EWZ: {indexed_prices.tickers['EWZ'].info['regularMarketPrice']}")

In [ ]:
patrimonial_prices = yf.Tickers(patrimonial_watchlist)

print(f" SE current Price: {patrimonial_prices.tickers['SE'].info['regularMarketPrice']}")
print(f" FRAGUAB.MX current Price: {patrimonial_prices.tickers['FRAGUAB.MX'].info['regularMarketPrice']}")
print(f" AGED.L current Price: {patrimonial_prices.tickers['AGED.L'].info['regularMarketPrice']}")

### Search 

In [ ]:
search_results = yf.Search(query="Sea Lim", max_results=7).response

In [ ]:
search_results

In [ ]:
## Printing only the needed info for each search result in UI
search_results
refined_results = []
for result in search_results['quotes']:
    refined_results.append({
        "ticker": result['symbol'],
        "displayed_name": result['shortname'],
        "exchange": result['exchange'],
        "asset_type": result['quoteType']
    })

refined_results

### Currency EXchange USD/MXN

In [ ]:
# Get currency tipo de cambio MXN/USD
tipo_cambio = yf.Lookup("USD/CHN").currency

In [ ]:
tipo_cambio

In [ ]:
tipo_cambio["shortName"]

In [ ]:
tipo_cambio["regularMarketPrice"].values

### Sector and Industry

In [ ]:
# Fetch Sector/industry information for a ticker
sector = yf.Sector('technology')
industry = yf.Industry('software-infrastructure')


In [ ]:

# Common information
print(sector.key)
print(sector.name)
print(sector.symbol)
print(sector.ticker.fast_info)
print(sector.overview)
print(sector.top_companies)
#tech.research_reports


In [ ]:
# Sector information

print(sector.top_etfs)
print(sector.top_mutual_funds)
print(sector.industries)

In [ ]:
# Industry information
print(industry.sector_key)
print(industry.sector_name)
print(industry.top_performing_companies)
print(industry.top_growth_companies)

Get Sector/Industry info from a particular ticker

In [ ]:
# Ticker to Sector and Industry
asset = yf.Ticker('MSTR')
sector_key = yf.Sector(asset.info.get('sectorKey'))
industry_key = yf.Industry(asset.info.get('industryKey'))

# Sector and Industry to Ticker
sector_ticker = sector_key.ticker
print(sector_ticker.info)


In [ ]:
industry_ticker = industry_key.ticker
industry_ticker.history()

### Single asset Price History

In [ ]:
## More detailed history, focused in one ticket in particular.
## Ticker.history() more meant to fundamental/technical analysis, not for price alerting, as it fetches a lot of data that is not needed for the use case.

last_year = seaholdings.history(period="1y", interval="1d")
last_year.head(50)


### Watchlists last month history

In [ ]:
# This is the most efficient way to get last month's price data,
# as it only fetches the data that is needed for the use case,
#  without fetching the whole ticker information.
# Gets last month up to yesterday close price.
indexed_prices = yf.download(indexed_watchlist, period="5d", interval="1d")

In [ ]:
indexed_prices.to_string()

In [ ]:
indexed_prices

In [ ]:
clean_prices = indexed_prices.drop(columns=["Volume"], level=0)
clean_prices = clean_prices.stack(level=1)
clean_prices = clean_prices.reset_index()
clean_prices = clean_prices.rename(columns={"level_1": "Ticker"})
clean_prices["Date"] = clean_prices["Date"].dt.strftime('%d-%m-%Y')
# clean_prices
grouped = clean_prices.groupby("Ticker")

result = {
    ticker: group.to_dict(orient="records") for ticker, group in grouped

}

result

### Other considerations


In case we have issues getting the latest price up to date, The Core Problem: Market Hours
The Most Optimal Approach for Your Use Case
For a scheduled job that needs the most up-to-date price (including pre-market or live session), the best approach is to use the .info dictionary. It's designed for exactly this kind of real-time data retrieval and is much lighter than downloading a full history every 15 minutes.

In [ ]:
import yfinance as yf
from datetime import datetime

def get_current_prices(tickers):
    """
    Gets the most current price for a list of tickers.
    It intelligently checks for pre-market, regular market, or previous close prices.

    :param tickers: A list of ticker symbols (e.g., ["AAPL", "MSFT"])
    :return: A dictionary mapping ticker to its current price.
    """
    prices = {}
    print(f"Fetching prices at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    for ticker_symbol in tickers:
        try:
            ticker = yf.Ticker(ticker_symbol)
            
            # .info is a dictionary containing a lot of real-time data
            info = ticker.info
            
            price = None
            
            # The order of checking is important to get the "most live" price
            if 'preMarketPrice' in info and info['preMarketPrice'] is not None:
                # Pre-market is active
                price = info['preMarketPrice']
            elif 'regularMarketPrice' in info and info['regularMarketPrice'] is not None:
                # Regular market is active
                price = info['regularMarketPrice']
            elif 'postMarketPrice' in info and info['postMarketPrice'] is not None:
                # Post-market is active
                price = info['postMarketPrice']
            else:
                # Fallback to the previous day's close if no live price is available
                price = info.get('previousClose')

            if price is not None:
                prices[ticker_symbol] = price
            else:
                prices[ticker_symbol] = "Price not available"

        except Exception as e:
            print(f"Could not get price for {ticker_symbol}: {e}")
            prices[ticker_symbol] = "Error"
            
    return prices

# --- How to use it in your scheduled job ---
my_tickers = ["AAPL", "MSFT", "NVDA", "TSLA"]
current_prices = get_current_prices(my_tickers)

print("\nCurrent Prices:")
for ticker, price in current_prices.items():
    print(f"{ticker}: {price}")


In [ ]:
##!! When a ticker name is wrong like FRAGUA.MX (should be FRAGUAB.MX), info object for any inexistent ticker is empty
## And yfinance does not throw an error, instead, it limits to print an HTTP error Not found in console, but it does not raise an exception that can be catched in code, so we need to check if
# the info object is empty to handle this case and return proper HTTP 404 error.


ticker_list = ["FRAGUA.MX", "SE", "VWO"]
indexed_prices = yf.Tickers(ticker_list)
try:
    fragua = indexed_prices.tickers[ticker_list[0]].info
    print(fragua)
except Exception as e:
    print(f"Error fetching info for {ticker_list[0]}: {e}")


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: FRAGUA.MX"}}}


{'trailingPegRatio': None}


In [37]:
prices = {ticker: indexed_prices.tickers[ticker].info['regularMarketPrice'] for ticker in ticker_list}
prices

NameError: name 'prices' is not defined